# Deep Learning based FACs and Action Unit occurrence and Intensity Estimation

\- Amogh Gudi

### Plan
1. Load dataset and labels
2. Build a model
3. Train Model
4. Testing Model

**Importing Libraries**

In [4]:
import keras
import numpy as np

Using TensorFlow backend.


### 1. Loading dataset and labels

### 2. Building a Model

In [38]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D,Flatten,Dropout
from keras.optimizers import SGD

In [34]:
model=Sequential()
model.add(Conv2D(64,kernel_size=(5,5),strides=(1,1),activation='relu',input_shape=(48,48,1)))
model.add(MaxPool2D(pool_size=(3,3),strides=(2,2)))
model.add(Conv2D(64,kernel_size=(5,5),strides=(1,1),activation='relu'))
model.add(Conv2D(128,kernel_size=(4,4),strides=(1,1),activation='relu'))
model.add(Flatten())
model.add(Dense(3072,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(6,activation='sigmoid'))

In [42]:
model.compile(loss='binary_crossentropy',optimizer=SGD(lr=0.01,momentum=0.9))

In [43]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_20 (Conv2D)           (None, 44, 44, 64)        1664      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 21, 21, 64)        0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 17, 17, 64)        102464    
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 14, 14, 128)       131200    
_________________________________________________________________
flatten_2 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 3072)              77073408  
_________________________________________________________________
dense_4 (Dense)              (None, 6)                 18438     
Total para

### 3. Training the model

### 4. Testing the model